# Calculating the reserve Factors of the panels 

# Please enter your name 

In [1]:
name = "yannis"

# Imports 

In [2]:
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))
sys.path.insert(0, os.path.abspath('../optimization'))

from formulas.panels import *
from formulas.helpers import *
from optimization.panelBuckReverse import *
from optimization.generation import *
 

In [3]:
# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])

# Constants (Changed based on your data)

In [4]:
personal_data = personal_data_provider(name)
sigma_yield = personal_data[0]
EModulus = personal_data[1]
nu = personal_data[2]
print(f"Your personal data is: sigma_yield = {sigma_yield}, EModulus = {EModulus}, nu = {nu}. Please verify.")

#Panel constants 
length = 750
width = 200


Your personal data is: sigma_yield = 490, EModulus = 65241.07, nu = 0.34. Please verify.


# Open stress values & properties of panels 

In [5]:
# New version of importing panel stress file
paneldf = pd.read_csv(f'../data/{name}/panel.csv')


#Import panel properties and reformat
panelPropertiesdf = pd.read_csv(f'../data/{name}/properties/panel_properties.csv', index_col=0)
panelPropertiesdf = panelPropertiesdf.reset_index()
panelPropertiesdf = panelPropertiesdf.drop(columns=['Component Name', 'mass'])


paneldf.head(5)


,Element ID,Component Name,sigmaXX,sigmaYY,sigmaXY,Load Case
0,1,panel1,-69.340,12.368,25.995,Subcase 1 (LC1)
1,2,panel1,-81.340,21.017,26.710,Subcase 1 (LC1)
2,3,panel1,-70.443,20.256,36.461,Subcase 1 (LC1)
3,4,panel2,-96.966,2.374,34.042,Subcase 1 (LC1)
4,5,panel2,-89.099,24.984,26.540,Subcase 1 (LC1)


# Split up the loadcases 

In [6]:
loadCase1df = paneldf[paneldf['Load Case'] == 'Subcase 1 (LC1)']
loadCase2df = paneldf[paneldf['Load Case'] == 'Subcase 2 (LC2)']
loadCase3df = paneldf[paneldf['Load Case'] == 'Subcase 3 (LC3)']
# Add the data to load case 
loadCase1df = pd.merge(loadCase1df, panelPropertiesdf, on='Element ID')
loadCase2df = pd.merge(loadCase2df, panelPropertiesdf, on='Element ID')
loadCase3df = pd.merge(loadCase3df, panelPropertiesdf, on='Element ID')

#Set index back to Element ID 
loadCase1df = loadCase1df.set_index('Element ID')
loadCase2df = loadCase2df.set_index('Element ID')
loadCase3df = loadCase3df.set_index('Element ID')

#Give some overview 
loadCase1df.head(5)

,Component Name,sigmaXX,sigmaYY,sigmaXY,Load Case,thickness
Element ID,,,,,,
1,panel1,-69.340,12.368,25.995,Subcase 1 (LC1),4.97
2,panel1,-81.340,21.017,26.710,Subcase 1 (LC1),4.97
3,panel1,-70.443,20.256,36.461,Subcase 1 (LC1),4.97
4,panel2,-96.966,2.374,34.042,Subcase 1 (LC1),5.19
5,panel2,-89.099,24.984,26.540,Subcase 1 (LC1),5.19


# Now we need to combine into panels:
-Panel ID (as our ID)
-sigmaXXavg
-sigmaYYavg
-sigmaXYavg
-length
-width
-thickness 


In [7]:
# We can just average the stresses @fabianober Tackle rounding!!!! 
#For load case 1 
panelLC1 = loadCase1df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'max'
})
dataframeLength = len(panelLC1)
panelLC1['length'] = [length]*dataframeLength
panelLC1['width'] = [width]*dataframeLength
panelLC1['Load Case'] = ['LC1'] * dataframeLength


#For load case 2 
panelLC2 = loadCase2df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'max'
})
dataframeLength = len(panelLC2)
panelLC2['length'] = [length]*dataframeLength
panelLC2['width'] = [width]*dataframeLength
panelLC2['Load Case'] = ['LC2'] * dataframeLength


#For load case 3
panelLC3 = loadCase3df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'max'
})
dataframeLength = len(panelLC3)
panelLC3['length'] = [length]*dataframeLength
panelLC3['width'] = [width]*dataframeLength
panelLC3['Load Case'] = ['LC3'] * dataframeLength


panelLC1

,sigmaXX,sigmaYY,sigmaXY,thickness,length,width,Load Case
Component Name,,,,,,,
panel1,-73.707667,17.880333,29.722000,4.97,750,200,LC1
panel10,-73.707667,17.880333,29.722000,4.97,750,200,LC1
panel2,-93.826667,17.879667,29.643333,5.19,750,200,LC1
panel3,-94.244667,17.880000,29.593333,4.98,750,200,LC1
panel4,-92.911667,17.880000,30.065000,5.45,750,200,LC1
panel5,-92.166667,17.880000,52.546333,6.55,750,200,LC1
panel6,-92.166667,17.880000,52.546333,6.55,750,200,LC1
panel7,-92.911667,17.880000,30.065000,5.45,750,200,LC1
panel8,-94.244667,17.880000,29.593333,4.98,750,200,LC1


# Now we can apply the functions to the load cases 

In [8]:
panelLC1[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC1.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC2[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC2.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC3[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC3.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC1

,sigmaXX,sigmaYY,sigmaXY,thickness,length,width,Load Case,k_shear,k_biaxial,Reserve Factor
Component Name,,,,,,,,,,
panel1,-73.707667,17.880333,29.722000,4.97,750,200,LC1,5.624444,5.026109,1.582573
panel10,-73.707667,17.880333,29.722000,4.97,750,200,LC1,5.624444,5.026109,1.582573
panel2,-93.826667,17.879667,29.643333,5.19,750,200,LC1,5.624444,4.824758,1.330840
panel3,-94.244667,17.880000,29.593333,4.98,750,200,LC1,5.624444,4.820475,1.214163
panel4,-92.911667,17.880000,30.065000,5.45,750,200,LC1,5.624444,4.834354,1.489082
panel5,-92.166667,17.880000,52.546333,6.55,750,200,LC1,5.624444,4.842322,2.061408
panel6,-92.166667,17.880000,52.546333,6.55,750,200,LC1,5.624444,4.842322,2.061408
panel7,-92.911667,17.880000,30.065000,5.45,750,200,LC1,5.624444,4.834354,1.489082
panel8,-94.244667,17.880000,29.593333,4.98,750,200,LC1,5.624444,4.820475,1.214163


# Combining back into 1 Data Frame for updating thickness 

In [9]:
updateParametersDf = pd.concat([panelLC1, panelLC2, panelLC3], axis=0)
updateParametersDf = updateParametersDf.reset_index()
evaluateDf = updateParametersDf.copy()
updateParametersDf

,Component Name,sigmaXX,sigmaYY,sigmaXY,thickness,length,width,Load Case,k_shear,k_biaxial,Reserve Factor
0,panel1,-73.707667,17.880333,29.722000,4.97,750,200,LC1,5.624444,5.026109,1.582573
1,panel10,-73.707667,17.880333,29.722000,4.97,750,200,LC1,5.624444,5.026109,1.582573
2,panel2,-93.826667,17.879667,29.643333,5.19,750,200,LC1,5.624444,4.824758,1.330840
3,panel3,-94.244667,17.880000,29.593333,4.98,750,200,LC1,5.624444,4.820475,1.214163
4,panel4,-92.911667,17.880000,30.065000,5.45,750,200,LC1,5.624444,4.834354,1.489082
5,panel5,-92.166667,17.880000,52.546333,6.55,750,200,LC1,5.624444,4.842322,2.061408
6,panel6,-92.166667,17.880000,52.546333,6.55,750,200,LC1,5.624444,4.842322,2.061408
7,panel7,-92.911667,17.880000,30.065000,5.45,750,200,LC1,5.624444,4.834354,1.489082
8,panel8,-94.244667,17.880000,29.593333,4.98,750,200,LC1,5.624444,4.820475,1.214163
9,panel9,-93.826667,17.879667,29.643333,5.19,750,200,LC1,5.624444,4.824758,1.330840


## Now we have to apply our reverse engineering and compute the thicknesses 

In [10]:
updateParametersDf["thickness"] = updateParametersDf.apply(panelBuckReverse,EModulus=EModulus, nu=nu, axis=1)
updateParametersDf

,Component Name,sigmaXX,sigmaYY,sigmaXY,thickness,length,width,Load Case,k_shear,k_biaxial,Reserve Factor
0,panel1,-73.707667,17.880333,29.722000,2.949825,750,200,LC1,5.624444,5.026109,1.582573
1,panel10,-73.707667,17.880333,29.722000,2.949825,750,200,LC1,5.624444,5.026109,1.582573
2,panel2,-93.826667,17.879667,29.643333,3.396888,750,200,LC1,5.624444,4.824758,1.330840
3,panel3,-94.244667,17.880000,29.593333,3.405958,750,200,LC1,5.624444,4.820475,1.214163
4,panel4,-92.911667,17.880000,30.065000,3.376927,750,200,LC1,5.624444,4.834354,1.489082
5,panel5,-92.166667,17.880000,52.546333,3.360593,750,200,LC1,5.624444,4.842322,2.061408
6,panel6,-92.166667,17.880000,52.546333,3.360593,750,200,LC1,5.624444,4.842322,2.061408
7,panel7,-92.911667,17.880000,30.065000,3.376927,750,200,LC1,5.624444,4.834354,1.489082
8,panel8,-94.244667,17.880000,29.593333,3.405958,750,200,LC1,5.624444,4.820475,1.214163
9,panel9,-93.826667,17.879667,29.643333,3.396888,750,200,LC1,5.624444,4.824758,1.330840


In [11]:
#Drop irrelevant columns
updateParametersDf = updateParametersDf.drop(['length', 'width'], axis=1)
updateParametersDf.head(3)


,Component Name,sigmaXX,sigmaYY,sigmaXY,thickness,Load Case,k_shear,k_biaxial,Reserve Factor
0,panel1,-73.707667,17.880333,29.722000,2.949825,LC1,5.624444,5.026109,1.582573
1,panel10,-73.707667,17.880333,29.722000,2.949825,LC1,5.624444,5.026109,1.582573
2,panel2,-93.826667,17.879667,29.643333,3.396888,LC1,5.624444,4.824758,1.330840


In [12]:
updateParametersDf = updateParametersDf.groupby('Component Name').agg({
    'thickness':'max'
})
updateParametersDf

,thickness
Component Name,
panel1,3.830948
panel10,3.830948
panel2,4.147132
panel3,4.159891
panel4,4.143422
panel5,4.700976
panel6,4.700976
panel7,4.143422
panel8,4.159891


# Score the corresponding parameter set 

In [13]:
evaluateDf = evaluateDf.drop(['sigmaXX', 'sigmaYY', 'sigmaXY', 'length', 'width', 'k_shear', 'k_biaxial'], axis =1)
evaluateDf['score'] = evaluateDf.apply(rf_score, axis=1)
evaluateDf

,Component Name,thickness,Load Case,Reserve Factor,score
0,panel1,4.97,LC1,1.582573,3053.373467
1,panel10,4.97,LC1,1.582573,3053.373467
2,panel2,5.19,LC1,1.330840,905.047578
3,panel3,4.98,LC1,1.214163,339.160896
4,panel4,5.45,LC1,1.489082,2107.565408
5,panel5,6.55,LC1,2.061408,10638.020914
6,panel6,6.55,LC1,2.061408,10638.020914
7,panel7,5.45,LC1,1.489082,2107.565408
8,panel8,4.98,LC1,1.214163,339.160896
9,panel9,5.19,LC1,1.330840,905.047578


## Extract score and parameters 

In [14]:
score = evaluateDf['score'].sum()
thicknesses = extractThickness(evaluateDf)

evaluateDf = pd.DataFrame({
    'panel thickness': [thicknesses],
    'score': [score],
    'minRF':[evaluateDf['Reserve Factor'].min()]
})
evaluateDf

,panel thickness,score,minRF
0,"[4.97, 5.19, 4.98, 5.45, 6.55]",70350.261094,0.828535


## Save score file 

In [15]:
evaluateDf = evaluateDf.round(rounding_digits)
evaluateDf.to_csv(f'../data/{name}/output/panelScore.csv', index = False)

# Create original output 

In [16]:
#Drop irrelevant columns
panelLC1 = panelLC1.drop(['length','thickness', 'width', 'Load Case'], axis=1)
panelLC2 = panelLC2.drop(['length','thickness', 'width',  'Load Case'], axis=1)
panelLC3 = panelLC3.drop(['length','thickness', 'width',  'Load Case'], axis=1)
panelLC1.head(3)

,sigmaXX,sigmaYY,sigmaXY,k_shear,k_biaxial,Reserve Factor
Component Name,,,,,,
panel1,-73.707667,17.880333,29.722000,5.624444,5.026109,1.582573
panel10,-73.707667,17.880333,29.722000,5.624444,5.026109,1.582573
panel2,-93.826667,17.879667,29.643333,5.624444,4.824758,1.330840


In [17]:
# Rename columns to be unambigous
panelLC1 = panelLC1.rename(columns={"Reserve Factor": "LC 1 RF", 'sigmaXX':'sigmaXXLC1', 'sigmaYY':'sigmaYYLC1', 'sigmaXY':'sigmaXYLC1', 'k_shear':'k_shearLC1', 'k_biaxial':'k_biaxialLC1'})
panelLC2 = panelLC2.rename(columns={"Reserve Factor": "LC 2 RF", 'sigmaXX':'sigmaXXLC2', 'sigmaYY':'sigmaYYLC2', 'sigmaXY':'sigmaXYLC2', 'k_shear':'k_shearLC2', 'k_biaxial':'k_biaxialLC2'})
panelLC3 = panelLC3.rename(columns={"Reserve Factor": "LC 3 RF", 'sigmaXX':'sigmaXXLC3', 'sigmaYY':'sigmaYYLC3', 'sigmaXY':'sigmaXYLC3', 'k_shear':'k_shearLC3', 'k_biaxial':'k_biaxialLC3'})
panelLC1

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,k_shearLC1,k_biaxialLC1,LC 1 RF
Component Name,,,,,,
panel1,-73.707667,17.880333,29.722000,5.624444,5.026109,1.582573
panel10,-73.707667,17.880333,29.722000,5.624444,5.026109,1.582573
panel2,-93.826667,17.879667,29.643333,5.624444,4.824758,1.330840
panel3,-94.244667,17.880000,29.593333,5.624444,4.820475,1.214163
panel4,-92.911667,17.880000,30.065000,5.624444,4.834354,1.489082
panel5,-92.166667,17.880000,52.546333,5.624444,4.842322,2.061408
panel6,-92.166667,17.880000,52.546333,5.624444,4.842322,2.061408
panel7,-92.911667,17.880000,30.065000,5.624444,4.834354,1.489082
panel8,-94.244667,17.880000,29.593333,5.624444,4.820475,1.214163


In [18]:
outputDf = pd.concat([panelLC1, panelLC2, panelLC3], axis=1)
outputDf

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,k_shearLC1,k_biaxialLC1,LC 1 RF,sigmaXXLC2,sigmaYYLC2,sigmaXYLC2,k_shearLC2,k_biaxialLC2,LC 2 RF,sigmaXXLC3,sigmaYYLC3,sigmaXYLC3,k_shearLC3,k_biaxialLC3,LC 3 RF
Component Name,,,,,,,,,,,,,,,,,,
panel1,-73.707667,17.880333,29.722000,5.624444,5.026109,1.582573,-76.008667,-17.880000,29.722000,5.624444,3.072999,0.966167,-24.486667,5.94,118.488667,5.624444,5.026098,1.103253
panel10,-73.707667,17.880333,29.722000,5.624444,5.026109,1.582573,-76.008667,-17.880000,29.722000,5.624444,3.072999,0.966167,-24.486667,5.94,118.488667,5.624444,5.026098,1.103253
panel2,-93.826667,17.879667,29.643333,5.624444,4.824758,1.330840,-93.875000,-17.880000,29.643333,5.624444,3.238663,0.907764,-31.170667,5.94,118.176000,5.624444,4.824774,1.201583
panel3,-94.244667,17.880000,29.593333,5.624444,4.820475,1.214163,-94.625667,-17.880000,29.593333,5.624444,3.244567,0.828535,-31.309667,5.94,117.976000,5.624444,4.820469,1.043226
panel4,-92.911667,17.880000,30.065000,5.624444,4.834354,1.489082,-93.657667,-17.879667,30.065000,5.624444,3.236955,1.005024,-30.866667,5.94,119.855000,5.624444,4.834353,1.396748
panel5,-92.166667,17.880000,52.546333,5.624444,4.842322,2.061408,-93.152333,-17.880000,52.546333,5.624444,3.232911,1.407375,-30.619333,5.94,209.480667,5.624444,4.842316,1.132782
panel6,-92.166667,17.880000,52.546333,5.624444,4.842322,2.061408,-93.152333,-17.880000,52.546333,5.624444,3.232911,1.407375,-30.619333,5.94,209.480667,5.624444,4.842316,1.132782
panel7,-92.911667,17.880000,30.065000,5.624444,4.834354,1.489082,-93.657667,-17.879667,30.065000,5.624444,3.236955,1.005024,-30.866667,5.94,119.855000,5.624444,4.834353,1.396748
panel8,-94.244667,17.880000,29.593333,5.624444,4.820475,1.214163,-94.625667,-17.880000,29.593333,5.624444,3.244567,0.828535,-31.309667,5.94,117.976000,5.624444,4.820469,1.043226


# ROUND & Output the files

In [19]:
outputDf = outputDf.round(rounding_digits)
outputDf.to_csv(f'../data/{name}/output/processed_e.csv')
outputDf.head(10)
updateParametersDf = updateParametersDf.round(rounding_digits)
updateParametersDf.to_csv(f'../data/{name}/output/updatePanelThick.csv')
updateParametersDf.head(10)

,thickness
Component Name,
panel1,3.83
panel10,3.83
panel2,4.15
panel3,4.16
panel4,4.14
panel5,4.70
panel6,4.70
panel7,4.14
panel8,4.16
